In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install transformers==3.1.0
!pip install transformers
!pip install accelerate
import torch
import transformers
import itertools
from tqdm.notebook import trange, tqdm

In [ ]:
model = transformers.BertModel.from_pretrained('bert-base-multilingual-cased',  device_map = 'cuda')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-cased',  device_map = 'cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
sents_folder = '/content/drive/MyDrive/MS IIITH/MS CourseWork/INLP/Project/Sentences'
urdu_sents_file = sents_folder+"/urdu_sents.txt"
telugu_sents_file = sents_folder+"/telugu_sents.txt"
tamil_sents_file = sents_folder+"/tamil_sents.txt"
hindi_sents_file = sents_folder+"/hindi_sents.txt"

In [ ]:
def read_file(file_path):
  f = open(file_path,'r',encoding='utf8')
  data = [l.strip() for l in f.readlines()]
  f.close()
  return data

# Generating alignements
Instruction: run the following cells seperately for each language i.e Hindi, Tamil, Telugu (change necessary variables and file names for saving)

In [ ]:
out = []
for  src,  tgt in tqdm(zip(read_file(urdu_sents_file),read_file(tamil_sents_file)), total=len(read_file(urdu_sents_file))):
  sent_src, sent_tgt = src.strip().split(), tgt.strip().split()
  token_src, token_tgt = [tokenizer.tokenize(word) for word in sent_src], [tokenizer.tokenize(word) for word in sent_tgt]
  wid_src, wid_tgt = [tokenizer.convert_tokens_to_ids(x) for x in token_src], [tokenizer.convert_tokens_to_ids(x) for x in token_tgt]
  ids_src, ids_tgt = tokenizer.prepare_for_model(list(itertools.chain(*wid_src)), return_tensors='pt', model_max_length=tokenizer.model_max_length, truncation=True)['input_ids'], tokenizer.prepare_for_model(list(itertools.chain(*wid_tgt)), return_tensors='pt', truncation=True, model_max_length=tokenizer.model_max_length)['input_ids']
  sub2word_map_src = []
  for i, word_list in enumerate(token_src):
    sub2word_map_src += [i for x in word_list]
  sub2word_map_tgt = []
  for i, word_list in enumerate(token_tgt):
    sub2word_map_tgt += [i for x in word_list]

  # alignment
  align_layer = 8
  threshold = 1e-3
  model.eval()
  with torch.no_grad():
    out_src = model(ids_src.unsqueeze(0).to("cuda"), output_hidden_states=True)[2][align_layer][0, 1:-1]
    out_tgt = model(ids_tgt.unsqueeze(0).to("cuda"), output_hidden_states=True)[2][align_layer][0, 1:-1]

    dot_prod = torch.matmul(out_src, out_tgt.transpose(-1, -2))

    softmax_srctgt = torch.nn.Softmax(dim=-1)(dot_prod)
    softmax_tgtsrc = torch.nn.Softmax(dim=-2)(dot_prod)

    softmax_inter = (softmax_srctgt > threshold)*(softmax_tgtsrc > threshold)

  align_subwords = torch.nonzero(softmax_inter, as_tuple=False)
  align_words = set()
  for i, j in align_subwords:
    align_words.add( (sub2word_map_src[i], sub2word_map_tgt[j]) )

  out.append({'Source Sentence':src, 'Target Sentence':tgt, 'Mapping/Alignment':sorted(align_words)})


  0%|          | 0/4248 [00:00<?, ?it/s]

In [ ]:
out[0]

{'Source Sentence': 'محمد السنوسی نے معمر قذافی کی مذمت اور وزیراعظم ڈیوڈ کیمرون فرانس کے صدر نکولس سرکوزی کو سراہا ہے کہ وہ لیبیا حکومت خلاف مجاہدین آزادی حمایت کر رہے ہیں ۔',
 'Target Sentence': 'முஹம்மது அல்-சுனுசி முயம்மர் கடாபியை கண்டித்தார் மற்றும் பிரதம மந்திரி டேவிட் கேமரூன் லிபிய அரசாங்கத்திற்கு எதிராக முஜாஹிதீன் ஆசாதியை ஆதரித்ததற்காக பிரெஞ்சு ஜனாதிபதி நிக்கோலஸ் சார்க்கோசியை பாராட்டினார் .',
 'Mapping/Alignment': [(0, 0),
  (1, 1),
  (2, 4),
  (3, 2),
  (3, 13),
  (4, 3),
  (5, 3),
  (6, 4),
  (7, 5),
  (8, 6),
  (8, 7),
  (9, 8),
  (10, 9),
  (11, 16),
  (14, 18),
  (15, 10),
  (15, 19),
  (16, 19),
  (17, 20),
  (18, 20),
  (21, 10),
  (23, 12),
  (23, 15),
  (24, 11),
  (24, 17),
  (26, 15),
  (27, 15),
  (30, 21)]}

In [ ]:
import pickle
# change the following 'output_pickle_file_path' variable for different languages
output_pickle_file_path = '/content/drive/MyDrive/MS IIITH/MS CourseWork/INLP/Project/map_files/urdu-tamil.pkl'
with open(output_pickle_file_path, 'wb') as output_pickle_file:
    pickle.dump(out, output_pickle_file)